# C-SVM

### 1. Introduction

In this notebook, we implemented a constrained SVM model based on the methologies introduced by the paper [A2-Maximizing accuracy under fairness constraints](https://arxiv.org/abs/1507.05259 ) to gain a better understanding of the trade-off between accuracy and fairness.

We used the [COMPAS dataset](https://github.com/propublica/compas-analysis/blob/master/compas-scores-two-years.csv), which contains the criminal history, jail and prison time, demographics, and COMPAS risk scores for defendants from Broward County from 2013 to 2014. 
* Binary class label (y): "two_year_recid" column, indicating whether the defendant recificated within two years
* Binary sensitive attribute (z): "race" column, Caucasian and African-American

### 2. Data Preprocessing

In [1]:
import numpy as np
from numpy.core.fromnumeric import transpose
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import scipy
from scipy.optimize import minimize
from numpy import linalg as LA

In [2]:
df = pd.read_csv('compas-scores-two-years.csv')
pd.set_option('display.max_columns', None)
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,violent_recid,is_violent_recid,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,2013-08-13,NaN,1.0,F,Aggravated Assault w/Firearm,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-08-14,Risk of Violence,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26,NaN,1.0,F,Felony Battery w/Prior Convict,1,13009779CF10A,(F3),NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,NaN,1,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),Risk of Recidivism,3,Low,2013-01-27,Risk of Violence,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,2013-04-13,NaN,1.0,F,Possession of Cocaine,1,13011511MM10A,(M1),0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,4,Low,2013-04-14,Risk of Violence,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,8,1,0,1,NaN,NaN,NaN,13000570CF10A,2013-01-12,NaN,1.0,F,Possession of Cannabis,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,8,High,2013-01-13,Risk of Violence,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,0,1,0,0,2,NaN,NaN,NaN,12014130CF10A,NaN,2013-01-09,76.0,F,arrest case no charge,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-03-26,Risk of Violence,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


We encoded the binary sensitive attribute such that African-American=1, Caucasian = 0.

In [3]:
df['race'] = df['race'].replace('African-American', 1).replace('Caucasian', 0)
df = df[(df['race'] == 0) | (df['race'] == 1)]

We also transformed four other attributes into more easily understandable and ready-to-use formats.

In [4]:
df['sex'] = df['sex'].replace('Male', 1).replace('Female', 0)
df['score_text'] = df['score_text'].replace('High', 1).replace('Medium', 0).replace('Low', -1)
df['c_charge_degree'] = df['c_charge_degree'].replace('M',1).replace('F',0)
df['days_in_jail'] = (pd.to_datetime(df['c_jail_out']) - pd.to_datetime(df['c_jail_in'])).dt.days

Our column selection is consistent with that of our LFR model, which is based on the correlation between each attribute.

In [6]:
X = df[['age', 'c_charge_degree', 'score_text', 'sex','c_days_from_compas','is_violent_recid','v_decile_score',
    'priors_count','juv_fel_count', 'juv_misd_count','juv_other_count','days_b_screening_arrest', 
   'decile_score', 'is_recid']]
Z = df['race']
y = df['two_year_recid']

Split the train and test sets. (training: testing =  2:1) 

In [7]:
X_train, X_test, Z_train, Z_test, y_train, y_test = train_test_split( X, Z, y, test_size=0.33, random_state=42)

Scale the training and testing data

In [8]:
scaler_train = preprocessing.StandardScaler().fit(X_train)
scaler_test = preprocessing.StandardScaler().fit(X_test)

### 3. Implementation

We implemented equation 9 from paper. Mathematically, we hope to minimize the loss function by solving the following quadratic program: 

<center> minimize $|\theta|^2 + C \sum_{i=1}^{n} \xi_i$
    <br>
 subject to $y_i \theta^T x_i \geq 1-\xi_i, \forall i \in {1,...,n}$
    <br>
 $\xi_i \geq 0, \forall i\in{1,...,n}$
    <br>
 $\frac{1}{N} \sum_{i=1}^{N} (Z_i - \bar{Z}) \theta^T x_i \leq c$
   <br>
  $\frac{1}{N} \sum_{i=1}^{N} (Z_i - \bar{Z}) \theta^T x_i \geq -c$

In [9]:
k = X_train.shape[1]
N = X_train.shape[0]

def upper_theta_constraint(params, X, Z, c, k):
    theta = params[:k]
    return ((-1/len(Z)) * np.matmul(np.matmul(transpose(Z - Z.mean()), X), theta)) + c

def lower_theta_constraint(params, X, Z, c, k):
    theta = params[:k]
    return ((np.matmul(np.matmul(transpose(Z - Z.mean()), X), theta))/len(Z)) + c  

def phi_constraint(params,k):
  #theta = params[:k]
    phi = params[k:]
    return phi 

def phi_constraint2(params, k, y , X):
    theta = params[:k]
    phi = params[k:]
    return np.dot( transpose(theta), np.matmul(y,X)) - 1 + sum(phi)

def svm_loss(params, X, y, C, k):
    theta = params[:k]
    phi = params[k:]
    y_hat = y * np.dot(X,theta) 
    y_hat = np.maximum(np.zeros_like(y_hat), (1-y_hat))     
    return C*sum(y_hat)

We use the [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) function from the scipy library to solve the optimization problem. 

In [10]:
theta = np.array(np.random.uniform(size=k)).reshape(-1, 1)
phi = np.array(np.random.uniform(size=N))
params = np.append(theta.flatten(), phi.flatten())
X_train_scaled = scaler_train.transform(X_train)

res = scipy.optimize.minimize(svm_loss, 
                              x0=params, 
                              args=(X_train_scaled, y_train, 0.8, k), 
                              method='SLSQP', 
                              constraints=({'type': 'ineq', 'fun': upper_theta_constraint, 'args': (X_train_scaled, Z_train , 0.8, k)},
                                     {'type': 'ineq', 'fun': lower_theta_constraint, 'args': (X_train_scaled, Z_train , 0.8, k)},
                                      {'type': 'ineq', 'fun': phi_constraint, 'args': [k]},
                                     {'type': 'ineq', 'fun': phi_constraint2, 'args': [k, y_train, scaler_train.transform(X_train)]}
                                    ))

We output the optimal parameters of the svm and save to a csv file. 

In [11]:
pd.DataFrame(res.x).to_csv('svm_parameters.csv')

In [12]:
k = X_train_scaled.shape[1]
N = X_train_scaled.shape[0]

params_hat = pd.read_csv('svm_parameters.csv')
theta_hat = params_hat['0'][0:k]

### 4. Evaluation

Using the retrieved theta parameter, we can generate the prediction for the testing data. 

In [13]:
X_test_scaled = scaler_test.transform(X_test)
yhat_test = np.matmul(theta_hat, np.transpose(X_test_scaled))

In [14]:
yhat_test[yhat_test < 0]

array([-0.11023027, -2.81416683, -0.50377019, ..., -0.15427302,
       -0.33559099, -0.17226149])

Now, we evaluate our model using the following 4 different metrics: 
- **Accuracy:** How often the model makes correct predictions.
- **Calibration:** Accuracy difference between two race groups 
- **Equality of odds:** Whether the odds of the positive outcome are equal for all groups or subpopulations being considered, regardless of other variables such as race, gender, or age.
- **Parity:** Whether the performance of the model is consistent across different subgroups of the population

#### Metric 1: Accuracy 

The overall accuracy of our model is 71%

In [15]:
acc = len(yhat_test[((yhat_test > 0) & (y_test > 0)) | ((yhat_test <= 0) & (y_test <= 0))])/len(yhat_test)
print('Accuracy of C-SVM: ', acc)

Accuracy of C-SVM:  0.7113300492610838


#### Metric 2: Calibration 

Our model is able to predict both groups with similar accuracy scores.

In [16]:
odds_pos = yhat_test[Z_test == 1]
y_pos = y_test[Z_test == 1]
acc_afr = len(odds_pos[((odds_pos > 0) & (y_pos > 0)) | ((odds_pos<= 0) & (y_pos <= 0))])/len(odds_pos)
print('Accuracy for African American: ', acc_afr)

odds_neg = yhat_test[Z_test == 0]
y_neg = y_test[Z_test == 0]
acc_cau= len(odds_neg[((odds_neg > 0) & (y_neg > 0)) | ((odds_neg <= 0) & (y_neg<= 0))])/len(odds_neg)
print('Accuracy for Caucasian: ', acc_cau)

print('Calibration: ', acc_afr - acc_cau)

Accuracy for African American:  0.7040065412919051
Accuracy for Caucasian:  0.7224287484510533
Calibration:  -0.01842220715914822


#### Metric 3: Equality of odds

Our model predicts the nonprotected group within the positive outcome with about 80% of the time, and protected group within the positive outcome with 47.5%. For negative target, it predicts 20% for nonprotected group and about 8% for protected group.

In [17]:
pos_afr = len(y_pos[(odds_pos > 0) & (y_pos > 0)]) / len(y_pos[odds_pos > 0])
pos_cau = len(y_pos[(odds_pos> 0) & (y_pos <= 0)]) / len(y_pos[y_pos <= 0])
print('Positive outcome - African American:', pos_afr)
print('Negative outcome - African American:', pos_cau)

neg_afr = len(y_neg[(odds_neg > 0) & (y_neg > 0)]) / len(y_neg[y_neg > 0])
neg_cau = len(y_neg[(odds_neg> 0) & (y_neg <= 0)]) / len(y_neg[y_neg <= 0])
print('Positive outcome - Caucasian:', neg_afr)
print('Negative outcome - Caucasian:', neg_cau)

Positive outcome - African American: 0.7996485061511424
Negative outcome - African American: 0.2
Positive outcome - Caucasian: 0.4752475247524752
Negative outcome - Caucasian: 0.07936507936507936


#### Metric 4: Parity

Our model predicts that 46% of nonprotected group will return to criminal behavior (positive outcome), while 22% of protected group will return to criminal behavior. 
We compared our prediction with the true outcomes, which is 53% vs. 37% respectively. 

In [18]:
#model outcome
pred_afr = len(odds_pos[odds_pos > 0])/len(odds_pos)
pred_cau = len(odds_neg[odds_neg > 0])/len(odds_neg)
print('Positive outcome - African American:',pred_afr)
print('Positive outcome - Caucasian:', pred_cau)

Positive outcome - African American: 0.4652493867538839
Positive outcome - Caucasian: 0.22800495662949194


In [19]:
#true values
true_afr = len(y_pos[y_pos > 0])/len(y_pos)
true_cau = len(y_neg[y_neg > 0])/len(y_neg)
print('Positive outcome - African American:',true_afr)
print('Positive outcome - Caucasian:', true_cau)

Positive outcome - African American: 0.5339329517579722
Positive outcome - Caucasian: 0.3754646840148699
